# TensorFlow BYOM: Train with Custom Training Script, Compile with Neo, and Deploy on SageMaker

This notebook can be compared to [TensorFlow MNIST distributed training notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_distributed_mnist/tensorflow_distributed_mnist.ipynb) in terms of its functionality. We will do the same classification task, but this time we will compile the trained model using the Neo API backend, to optimize for our choice of hardware. Finally, we setup a real-time hosted endpoint in SageMaker for our compiled model using the Neo Deep Learning Runtime.

### Set up the environment

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### Download the MNIST dataset

In [2]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Writing data/train.tfrecords

Writing data/validation.tfrecords
Writing data/test.tfrecords


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [3]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')

# Construct a script for distributed training 
Here is the full code for the network model:

In [5]:
!cat 'mnist.py'

import os
import tensorflow as tf
from tensorflow.python.estimator.model_fn import ModeKeys as Modes

INPUT_TENSOR_NAME = 'inputs'
SIGNATURE_NAME = 'predictions'

LEARNING_RATE = 0.001


def model_fn(features, labels, mode, params):
    # Input Layer
    input_layer = tf.reshape(features[INPUT_TENSOR_NAME], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.d

The script here is and adaptation of the [TensorFlow MNIST example](https://github.com/tensorflow/models/tree/master/official/mnist). It provides a ```model_fn(features, labels, mode)```, which is used for training, evaluation and inference. See [TensorFlow MNIST distributed training notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_distributed_mnist/tensorflow_distributed_mnist.ipynb) for more details about the training script.

At the end of the training script, there are two additional functions, to be used with Neo Deep Learning Runtime:
* `neo_preprocess(payload, content_type)`: Function that takes in the payload and Content-Type of each incoming request and returns a NumPy array
* `neo_postprocess(result)`: Function that takes the prediction results produced by Deep Learining Runtime and returns the response body

## Create a training job using the sagemaker.TensorFlow estimator

In [6]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             framework_version='1.11.0',
                             training_steps=1000, 
                             evaluation_steps=100,
                             train_instance_count=1,
                             train_instance_type='ml.c4.xlarge')

mnist_estimator.fit(inputs)

tensorflow py2 container will be deprecated soon.


2020-02-04 21:53:01 Starting - Starting the training job...
2020-02-04 21:53:03 Starting - Launching requested ML instances......
2020-02-04 21:54:08 Starting - Preparing the instances for training...
2020-02-04 21:55:01 Downloading - Downloading input data
2020-02-04 21:55:01 Training - Downloading the training image..2020-02-04 21:55:13,519 INFO - root - running container entrypoint
2020-02-04 21:55:13,519 INFO - root - starting train task
2020-02-04 21:55:13,532 INFO - container_support.training - Training starting

2020-02-04 21:55:13 Training - Training image download completed. Training in progress.2020-02-04 21:55:31,589 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2020-02-04 21:55:31,589 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2020-02-04 21:55:31,590 INFO - tf_container - ---------------------------------------------------------
2020-02-04 21:55:31,590 INFO 

The **```fit```** method will create a training job in two **ml.c4.xlarge** instances. The logs above will show the instances doing training, evaluation, and incrementing the number of **training steps**. 

In the end of the training, the training job will generate a saved model for TF serving.

# Deploy the trained model to prepare for predictions (the old way)

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [7]:
mnist_predictor = mnist_estimator.deploy(initial_instance_count=1,
                                         instance_type='ml.m4.xlarge')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


---------------------------------------------------------------------------------------------------!

## Invoking the endpoint

In [8]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i].tolist()
    tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)
    predict_response = mnist_predictor.predict(tensor_proto)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response['outputs']['classes']['int64_val'][0]
    print("prediction is {}".format(prediction))

Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
label is 7
prediction is 7
label is 2
prediction is 2
label is 1
prediction is 1
label is 0
prediction is 0
label is 4
prediction is 4
label is 1
prediction is 1
label is 4
prediction is 4
label is 9
prediction is 9
label is 5
prediction is 5
label is 9
prediction is 9


## Deleting the endpoint

In [9]:
sagemaker.Session().delete_endpoint(mnist_predictor.endpoint)

# Deploy the trained model using Neo

Now the model is ready to be compiled by Neo to be optimized for our hardware of choice. We are using the  ``TensorFlowEstimator.compile_model`` method to do this. For this example, our target hardware is ``'ml_c5'``. You can changed these to other supported target hardware if you prefer.

## Compiling the model
The ``input_shape`` is the definition for the model's input tensor and ``output_path`` is where the compiled model will be stored in S3. **Important. If the following command result in a permission error, scroll up and locate the value of execution role returned by `get_execution_role()`. The role must have access to the S3 bucket specified in ``output_path``.**

In [10]:
output_path = '/'.join(mnist_estimator.output_path.split('/')[:-1])
optimized_estimator = mnist_estimator.compile_model(target_instance_family='ml_c5', 
                              input_shape={'data':[1, 784]},  # Batch size 1, 3 channels, 224x224 Images.
                              output_path=output_path,
                              framework='tensorflow', framework_version='1.11.0')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


?...........!

## Deploying the compiled model

In [11]:
optimized_predictor = optimized_estimator.deploy(initial_instance_count = 1,
                                                 instance_type = 'ml.c5.4xlarge')

----------------------------------------------------------------------------------------------------!

In [12]:
def numpy_bytes_serializer(data):
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

optimized_predictor.content_type = 'application/vnd+python.numpy+binary'
optimized_predictor.serializer = numpy_bytes_serializer

## Invoking the endpoint

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
from IPython import display
import PIL.Image
import io

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i]
    # Display image
    im = PIL.Image.fromarray(data.reshape((28,28))*255).convert('L')
    display.display(im)
    # Invoke endpoint with image
    predict_response = optimized_predictor.predict(data)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response
    print("prediction is {}".format(prediction))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


label is 7
prediction is [0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.23196931183338165, 0.08533674478530884, 0.08533674478530884]


label is 2
prediction is [0.08533674478530884, 0.08533674478530884, 0.23196931183338165, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884]


label is 1
prediction is [0.08533674478530884, 0.23196931183338165, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884]


label is 0
prediction is [0.23196931183338165, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884]


...

label is 1
prediction is [0.08533674478530884, 0.23196931183338165, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884]


label is 4
prediction is [0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.23196931183338165, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884]


label is 9
prediction is [0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.23196931183338165]


label is 5
prediction is [0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.23196931183338165, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884]


label is 9
prediction is [0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.08533674478530884, 0.23196931183338165]


## Deleting endpoint

In [14]:
sagemaker.Session().delete_endpoint(optimized_predictor.endpoint)